In [1]:
using DifferentialEquations
#using ADCME
using Plots

In [167]:
function f_hi(a, T, R)
        v1 = (a[:,2] + a[:,3] * T + a[:,4] * T^2 + a[:,5] * T^3 + a[:,6] * T^4) * R
        v2 = (a[:,2] + a[:,3] * T / 2 + a[:,4] * T^2 /3 + a[:,5] * T^3 /4 + a[:,6] * T^4 /5 + a[:,7] / T) * R * T
        v3 = (a[:,2] * log(T) + a[:,3] * T + a[:,4] * T^2 /2 + a[:,5] * T^3 /3 + a[:,6] * T^4 / 4 + a[:,8]) * R
    return [v1 v2 v3]
end
 
function f_lo(a, T, R)
    v1 = (a[:,9] + a[:,10] * T + a[:,11] * T^2 + a[:,12] * T^3 + a[:,13] * T^4) * R
    v2 = (a[:,9] + a[:,10] * T / 2 + a[:,11] * T^2 /3 + a[:,12] * T^3 /4 + a[:,13] * T^4 /5 + a[:,14]/T) * R * T
    v3 = (a[:,9] * log(T) + a[:,10] * T + a[:,11] * T^2 /2 + a[:,12] * T^3 /3 + a[:,13] * T^4 / 4 + a[:,15]) * R
    return [v1 v2 v3]
end 

function compute_falloff(T, pr, a)
    fcent = (1 - a[1]) * exp(-T/a[2]) + a[1] * exp(-T/a[3]) + exp(-a[4]/T)
    c = -0.4 - 0.67 * log(10, fcent)
    n = 0.75 - 1.27 * log(10, fcent)
    f1 = (log(10, pr) + c) / (n - 0.14 *(log(10, pr) + c))
    return  10 ^ (log(10, fcent) / (1 + f1 ^ 2))
end

compute_falloff (generic function with 1 method)

In [168]:
using PyCall
#### Read data
py"""
import numpy as np
"""

In [176]:
V = 1  # Volume
m = (py"np.load"("data/density.npy"))[1] # Total mass
tbd = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/tbd.npy") .+ 1
tbd = Int.(tbd)
falofr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/falofr.npy") .+ 1
falofr = Int.(falofr)
elmr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/elmr.npy") .+ 1
elmr = Int.(elmr)
NASA_coeffs = py"np.load"("data/NASA_coeffs.npy")
W = py"np.load"("data/molecular_weights.npy") # Molar weight
ν1 = py"np.load"("data/reactants_stoich_coeffs.npy") # Forward molar stoichiometric coefficients
ν2 = py"np.load"("data/product_stoich_coeffs.npy") # Backward model stoichiometric coefficients
reversible = py"np.load"("data/reversible.npy")
N = size(ν1)[1]  # Number of Species
M = size(ν1)[2]  # Number of Reactions
ν1_order = zeros(N,M)
ν = ν2 - ν1  
pa = 100000 # 1 bar
R = 8314.4621 # Gas constant in kmol
### Constants: Combustion Chamber Level
min_dot = 0 # Rate at which mass enters the chamber
mout_dot = 0 # Rate at which mass leaves the chamber
Yin = ones(N) # Mass fraction of species entering the chamber
Yout = ones(N) # Mass fraction of species leaving the chamber
Qdot = 0 # Heating source
m_dot = min_dot - mout_dot
hin = 1 # Enthalpy of input
### Unknowns
Tt_cant = py"np.load"("data/temperature.npy")
T = Tt_cant[1] # Temperature
Y = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/ini_mass_fraction.npy") # Mass fractions
### Quantities that depend on the Unknowns
order = py"np.load"("data/reaction_orders.npy")
Af = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor.npy") # preexponential constant Afj
β = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent.npy") # Temperature exponent
E = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy.npy") # Activation energy for the reactions in kJ

order_t = py"np.load"("data/reaction_orders_t.npy")
efficiency_t = py"np.load"("data/efficiency_t.npy")
Af_t = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor_t.npy") # preexponential constant Afj
β_t = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent_t.npy") # Temperature exponent
E_t = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy_t.npy") # Activation energy for the reactions in kJ

order_f = py"np.load"("data/reaction_orders_f.npy")
troefall = py"np.load"("data/troefall.npy") .+ 1
troefall_coeff = py"np.load"("data/troefall_coeff.npy")
efficiency_f = py"np.load"("data/efficiency_f.npy")
Af_hi = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor_hi.npy") # preexponential constant Afj
β_hi = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent_hi.npy") # Temperature exponent
E_hi = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy_hi.npy") 

Af_lo = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor_lo.npy") # preexponential constant Afj
β_lo = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent_lo.npy") # Temperature exponent
E_lo = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy_lo.npy") 

ν1_order[:,elmr] = order
ν1_order[:,tbd] = order_t
ν1_order[:,falofr] = order_f


6×0 Array{Float64,2}

In [177]:
ρ = m / V # density
X = ρ * Y ./ W # Concentration
Q = ones(M) # Individual progress rates
# cpHS = compute_cpHS(NASA_coeffs, R, T)
# cp = cpHS[:,1]
# h = cpHS[:,2]
# s = cpHS[:,3]
islarge = NASA_coeffs[:,1] .< T * ones(N)
fhi = f_hi(NASA_coeffs, T, R)
flo = f_lo(NASA_coeffs, T, R)
cp = islarge .* fhi[:,1] + (1 .- islarge) .* flo[:,1]
h = islarge .* fhi[:,2] + (1 .- islarge) .* flo[:,2]
s = islarge .* fhi[:,3] + (1 .- islarge) .* flo[:,3]
cvk = cp .- R
ΔS = ν' * s  # Entropy change for reaction j
ΔH = ν' * h # Entahlpy change for reaction j
####
M_t = efficiency_t' * X
Kf_t = Af_t .* (T .^ β_t) .* exp.(-E_t ./ (R * T)) .* M_t
Kr_t = Kf_t ./ (((pa/(R * T)) .^ sum(ν[:,tbd], dims=1)' .* exp.(ΔS[tbd] ./ R - ΔH[tbd] ./ (R * T))))
Qtbd = Kf_t .* prod(X .^ order_t, dims=1)' .- Kr_t .* prod(X .^ ν2[:,tbd], dims=1)' .* reversible[tbd]
####
M_f = efficiency_f' * X
Kf_lo = Af_lo .* (T .^ β_lo) .* exp.(-E_lo ./ (R * T)) .* M_f
Kf_hi = Af_hi .* (T .^ β_hi) .* exp.(-E_hi ./ (R * T)) 
Pr = Kf_lo ./ Kf_hi
Fac = ones(size(falofr)[1])
Fac[troefall] = [compute_falloff(T, Pr[s], troefall_coeff[:,i]) for (i,s) in enumerate(troefall)]
Kf_f = Kf_lo ./ (1 .+ (Kf_lo ./ Kf_hi)) .* Fac
Kr_f = Kf_f ./ (((pa/(R * T)) .^ sum(ν[:,falofr], dims=1)' .* exp.(ΔS[falofr] ./ R - ΔH[falofr] ./ (R * T))))
Qfalofr = Kf_f .* prod(X .^ order_f, dims=1)' .- Kr_f .* prod(X .^ ν2[:,falofr], dims=1)' .* reversible[falofr]
####
Kf = Af .* (T .^ β) .* exp.(-E ./ (R * T))
Kr = Kf ./ (((pa/(R * T)) .^ sum(ν[:,elmr], dims=1)' .* exp.(ΔS[elmr] ./ R - ΔH[elmr] ./ (R * T))))
Qelmr = Kf .* prod(X .^ order, dims=1)' .- Kr .* prod(X .^ ν2[:,elmr], dims=1)' .* reversible[elmr]
##### Computing ω_dot 
cv = sum(cvk ./ W .* Y) # Mass heat capacities
u = h ./ W - R ./ W .* T   # Internal energy for species
p = sum(X) * R * T # pressure
Q = [Qtbd' Qfalofr' Qelmr']
ν_new = [ν[:,tbd] ν[:,falofr] ν[:,elmr]]
ω_dot = W .* sum(ν_new .* Q, dims=2)
###### Species Conservation
mgen_dot = V .* ω_dot
Y_dot = (1 / m) .* (min_dot .* (Yin .- Y) - mout_dot .* Y .+ mgen_dot) 
###### Energy Conservation
T_dot = 1 / (m * cv) * (-Qdot + min_dot * (hin - sum(u .* Yin)) - p * V / m * mout_dot - sum(mgen_dot .* u))

29251.07568439282

In [178]:
function q(t)
    t0 = 1e-5
    r = 1e-6
    c = 0
    return c * exp(-(t-t0)^2 / r^2)
end

q (generic function with 1 method)

In [179]:
function f(TY, t)
    T = TY[1]
    Y = TY[2:end]
    ρ = m / V # density
    X = ρ * Y ./ W # Concentration
    Q = ones(M) # Individual progress rates
    islarge = NASA_coeffs[:,1] .< T * ones(N)
    fhi = f_hi(NASA_coeffs, T, R)
    flo = f_lo(NASA_coeffs, T, R)
    cp = islarge .* fhi[:,1] + (1 .- islarge) .* flo[:,1]
    h = islarge .* fhi[:,2] + (1 .- islarge) .* flo[:,2]
    s = islarge .* fhi[:,3] + (1 .- islarge) .* flo[:,3]
    cvk = cp .- R
    ΔS = ν' * s  # Entropy change for reaction j
    ΔH = ν' * h # Entahlpy change for reaction j
    ####
    M_t = efficiency_t' * X
    Kf_t = Af_t .* (T .^ β_t) .* exp.(-E_t ./ (R * T)) .* M_t
    Kr_t = Kf_t ./ (((pa/(R * T)) .^ sum(ν[:,tbd], dims=1)' .* exp.(ΔS[tbd] ./ R - ΔH[tbd] ./ (R * T))))
    Qtbd = Kf_t .* prod(X .^ order_t, dims=1)' .- Kr_t .* prod(X .^ ν2[:,tbd], dims=1)' .* reversible[tbd]
    ####
    M_f = efficiency_f' * X
    Kf_lo = Af_lo .* (T .^ β_lo) .* exp.(-E_lo ./ (R * T)) .* M_f
    Kf_hi = Af_hi .* (T .^ β_hi) .* exp.(-E_hi ./ (R * T)) 
    Pr = Kf_lo ./ Kf_hi
    Fac = ones(size(falofr)[1])
    Fac[troefall] = [compute_falloff(T, Pr[s], troefall_coeff[:,i]) for (i,s) in enumerate(troefall)]
    Kf_f = Kf_lo ./ (1 .+ (Kf_lo ./ Kf_hi)) .* Fac
    Kr_f = Kf_f ./ (((pa/(R * T)) .^ sum(ν[:,falofr], dims=1)' .* exp.(ΔS[falofr] ./ R - ΔH[falofr] ./ (R * T))))
    Qfalofr = Kf_f .* prod(X .^ order_f, dims=1)' .- Kr_f .* prod(X .^ ν2[:,falofr], dims=1)' .* reversible[falofr]
    ####
    Kf = Af .* (T .^ β) .* exp.(-E ./ (R * T))
    Kr = Kf ./ (((pa/(R * T)) .^ sum(ν[:,elmr], dims=1)' .* exp.(ΔS[elmr] ./ R - ΔH[elmr] ./ (R * T))))
    Qelmr = Kf .* prod(X .^ order, dims=1)' .- Kr .* prod(X .^ ν2[:,elmr], dims=1)' .* reversible[elmr]
    ##### Computing ω_dot 
    cv = sum(cvk ./ W .* Y) # Mass heat capacities
    u = h ./ W - R ./ W .* T   # Internal energy for species
    p = sum(X) * R * T # pressure
    Q = [Qtbd' Qfalofr' Qelmr']
    ν_new = [ν[:,tbd] ν[:,falofr] ν[:,elmr]]
    ω_dot = W .* sum(ν_new .* Q, dims=2)
    ###### Species Conservation
    mgen_dot = V .* ω_dot
    Y_dot = (1 / m) .* (min_dot .* (Yin .- Y) - mout_dot .* Y .+ mgen_dot) 
    ###### Energy Conservation
    T_dot = 1 / (m * cv) * (q(t) + min_dot * (hin - sum(u .* Yin)) - p * V / m * mout_dot - sum(mgen_dot .* u))
    return [T_dot; Y_dot]
end

f (generic function with 2 methods)

In [180]:
timestep

50001-element Array{Float64,1}:
 0.0
 1.0e-11
 3.0e-11
 6.0e-11
 9.999999999999999e-11
 1.5e-10
 2.1e-10
 2.7999999999999996e-10
 3.5999999999999995e-10
 4.4999999999999995e-10
 5.5e-10
 6.6e-10
 7.8e-10
 ⋮
 0.01249475055
 0.012495250449999998
 0.012495750359999998
 0.01249625028
 0.01249675021
 0.01249725015
 0.0124977501
 0.01249825006
 0.01249875003
 0.01249925001
 0.01249975
 0.012500250000000001

In [181]:
## RK4
#dt = 4e-9
timestep = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/timestep.npy")
n_t = size(timestep)[1]
#n_t = 2000
T = Tt_cant[1] # Temperature
Y = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/ini_mass_fraction.npy") # Mass fractions
TY = [T;Y]
TYt = zeros(N+1, n_t)
Tt = zeros(n_t)
Pt = zeros(n_t)
Yt = zeros(N,n_t)
Y0 = copy(Y)
Yt[:,1] .= Y
Tt[1] = T
X = ρ * Y ./ W
Pt[1] = sum(X) * R * T
for i = 1:n_t - 1
    dt = timestep[i+1] - timestep[i]
    k1 = f(TY, timestep[i])
    k2 = f(TY + 0.5 * dt * k1, timestep[i] + 0.5 * dt)
    k3 = f(TY + 0.5 * dt * k2, timestep[i] + 0.5 * dt)
    k4 = f(TY + dt * k3, timestep[i] + dt)
    TY += 1/6 * dt * (k1 + 2 * k2 + 2 * k3 + k4)
    #TY += dt * f(TY)
    Tt[i+1] = TY[1]
    println(TY[1])
    println(i)
    Y = TY[2:end]
    Yt[:,i+1] = Y
    X = ρ * Y ./ W
    TYt[:,i+1] = TY
    Pt[i+1] = sum(X) * R * TY[1]
end



500.00000005850217
1
500.00000011700433
2
500.00000020475755
3
500.00000032176183
4
500.0000004680172
5
500.00000064352366
6
500.0000008482812
7
500.0000010822898
8
500.0000013455495
9
500.00000163806027
10
500.0000019598221
11
500.00000231083504
12
500.00000269109904
13
500.00000310061415
14
500.0000035393803
15
500.0000040073976
16
500.0000045046659
17
500.00000503118537
18
500.00000558695587
19
500.0000061719775
20
500.00000678625014
21
500.0000074297739
22
500.00000810254875
23
500.0000088045747
24
500.00000953585175
25
500.00001029637986
26
500.0000110861591
27
500.0000119051894
28
500.00001275347086
29
500.0000136310034
30
500.000014537787
31
500.00001547382175
32
500.0000164391076
33
500.00001743364453
34
500.0000184574326
35
500.00001951047176
36
500.00002059276204
37
500.00002170430344
38
500.00002284509594
39
500.00002401513956
40
500.00002521443434
41
500.00002644298024
42
500.00002770077725
43
500.0000289878254
44
500.00003030412466
45
500.00003164967507
46
500.000033024476

500.0041473810951
532
500.0041629733711
533
500.00417859490636
534
500.00419424570094
535
500.0042099257548
536
500.0042256350681
537
500.00424137364075
538
500.0042571414728
539
500.0042729385643
540
500.0042887649153
541
500.00430462052583
542
500.0043205053959
543
500.0043364195256
544
500.00435236291486
545
500.00436833556375
546
500.00438433747235
547
500.00440036864063
548
500.00441642906867
549
500.00443251875646
550
500.00444863770406
551
500.00446478591147
552
500.00448096337874
553
500.0044971701059
554
500.004513406093
555
500.00452967134004
556
500.00454596584706
557
500.0045622896141
558
500.0045786426412
559
500.0045950249284
560
500.0046114364757
561
500.0046278772832
562
500.00464434735085
563
500.0046608466787
564
500.0046773752668
565
500.00469393311516
566
500.00471052022385
567
500.0047271365929
568
500.00474378222236
569
500.0047604571122
570
500.0047771612625
571
500.00479389467324
572
500.00481065734454
573
500.0048274492763
574
500.0048442704687
575
500.00486112

500.01541365098643
1026
500.0154437022355
1027
500.015473782766
1028
500.015503892578
1029
500.0155340316716
1030
500.01556420004687
1031
500.01559439770386
1032
500.0156246246426
1033
500.0156548808632
1034
500.0156851663657
1035
500.01571548115015
1036
500.0157458252166
1037
500.0157761985651
1038
500.0158066011957
1039
500.01583703310854
1040
500.0158674943036
1041
500.015897984781
1042
500.0159285045408
1043
500.015959053583
1044
500.0159896319077
1045
500.01602023951494
1046
500.0160508764048
1047
500.0160815425773
1048
500.0161122380325
1049
500.0161429627705
1050
500.0161737167914
1051
500.01620450009517
1052
500.01623531268194
1053
500.0162661545517
1054
500.0162970257046
1055
500.01632792614066
1056
500.0163588558599
1057
500.0163898148624
1058
500.01642080314826
1059
500.0164518207175
1060
500.01648286757023
1061
500.01651394370646
1062
500.0165450491263
1063
500.0165761838298
1064
500.01660734781694
1065
500.0166385410879
1066
500.01666976364265
1067
500.0167010154813
1068
5

500.02792222135685
1381
500.02796267167827
1382
500.0280031513058
1383
500.0280436602396
1384
500.02808419847963
1385
500.02812476602605
1386
500.0281653628789
1387
500.02820598903827
1388
500.0282466445043
1389
500.02828732927696
1390
500.02832804335645
1391
500.02836878674276
1392
500.028409559436
1393
500.0284503614363
1394
500.0284911927437
1395
500.0285320533582
1396
500.02857294328
1397
500.02861386250913
1398
500.02865481104567
1399
500.0286957888897
1400
500.0287367960413
1401
500.02877783250057
1402
500.02881889826756
1403
500.0288599933424
1404
500.0289011177251
1405
500.0289422714158
1406
500.0289834544146
1407
500.0290246667215
1408
500.02906590833663
1409
500.02910717926005
1410
500.0291484794919
1411
500.0291898090322
1412
500.02923116788105
1413
500.0292725560385
1414
500.0293139735047
1415
500.0293554202797
1416
500.0293968963636
1417
500.0294384017564
1418
500.02947993645824
1419
500.02952150046923
1420
500.0295630937894
1421
500.0296047164189
1422
500.0296463683577
14

500.0500028094183
1848
500.05005695535857
1849
500.05011113064853
1850
500.0501653352883
1851
500.05021956927794
1852
500.0502738326176
1853
500.05032812530743
1854
500.05038244734743
1855
500.05043679873773
1856
500.0504911794785
1857
500.0505455895698
1858
500.0506000290118
1859
500.0506544978045
1860
500.05070899594807
1861
500.0507635234426
1862
500.0508180802882
1863
500.05087266648496
1864
500.05092728203306
1865
500.05098192693254
1866
500.0510366011835
1867
500.0510913047861
1868
500.0511460377404
1869
500.05120080004656
1870
500.05125559170466
1871
500.0513104127148
1872
500.0513652630771
1873
500.0514201427917
1874
500.05147505185863
1875
500.051529990278
1876
500.05158495805006
1877
500.0516399551748
1878
500.0516949816523
1879
500.05175003748275
1880
500.0518051226662
1881
500.0518602372028
1882
500.0519153810926
1883
500.0519705543358
1884
500.05202575693244
1885
500.0520809888826
1886
500.0521362501865
1887
500.05219154084415
1888
500.0522468608557
1889
500.0523022102212


...

506.638763948355
20543
506.63945986417286
20544
506.6401558244174
20545
506.6408518290904
20546
506.6415478781938
20547
506.64224397172944
20548
506.64294010969905
20549
506.64363629210453
20550
506.6443325189477
20551
506.64502879023036
20552
506.64572510595434
20553
506.64642146612147
20554
506.6471178707336
20555
506.6478143197926
20556
506.6485108133002
20557
506.6492073512583
20558
506.6499039336688
20559
506.6506005605334
20560
506.651297231854
20561
506.65199394763243
20562
506.6526907078705
20563
506.65338751257
20564
506.6540843617329
20565
506.6547812553609
20566
506.65547819345585
20567
506.65617517601964
20568
506.6568722030541
20569
506.657569274561
20570
506.6582663905423
20571
506.65896355099966
20572
506.659660755935
20573
506.6603580053502
20574
506.66105529924704
20575
506.66175263762733
20576
506.66245002049294
20577
506.66314744784574
20578
506.6638449196875
20579
506.6645424360201
20580
506.66523999684534
20581
506.6659376021651
20582
506.6666352519812
20583
506.66

506.9757926262493
21020
506.97650994437174
21021
506.9772273078131
21022
506.9779447165753
21023
506.9786621706602
21024
506.97937967006976
21025
506.98009721480594
21026
506.9808148048706
21027
506.9815324402656
21028
506.98225012099294
21029
506.9829678470545
21030
506.9836856184522
21031
506.98440343518797
21032
506.9851212972637
21033
506.98583920468127
21034
506.98655715744263
21035
506.9872751555497
21036
506.9879931990044
21037
506.98871128780866
21038
506.98942942196436
21039
506.99014760147344
21040
506.99086582633777
21041
506.99158409655934
21042
506.99230241214
21043
506.99302077308175
21044
506.9937391793864
21045
506.9944576310559
21046
506.9951761280922
21047
506.99589467049725
21048
506.9966132582729
21049
506.997331891421
21050
506.99805056994364
21051
506.9987692938426
21052
506.9994880631199
21053
507.0002068777774
21054
507.000925737817
21055
507.00164464324064
21056
507.0023635940502
21057
507.0030825902477
21058
507.00380163183496
21059
507.004520718814
21060
507.

507.2333024311703
21375
507.2340359590127
21376
507.2347695328644
21377
507.23550315272723
21378
507.23623681860323
21379
507.2369705304944
21380
507.23770428840265
21381
507.23843809233
21382
507.23917194227846
21383
507.23990583824997
21384
507.2406397802465
21385
507.2413737682701
21386
507.2421078023226
21387
507.2428418824061
21388
507.24357600852255
21389
507.24431018067395
21390
507.2450443988622
21391
507.24577866308937
21392
507.2465129733574
21393
507.2472473296683
21394
507.247981732024
21395
507.2487161804265
21396
507.24945067487783
21397
507.2501852153799
21398
507.2509198019347
21399
507.2516544345442
21400
507.2523891132105
21401
507.25312383793545
21402
507.2538586087211
21403
507.25459342556934
21404
507.25532828848225
21405
507.2560631974618
21406
507.25679815250993
21407
507.25753315362863
21408
507.2582682008199
21409
507.2590032940857
21410
507.259738433428
21411
507.2604736188489
21412
507.2612088503503
21413
507.26194412793416
21414
507.2626794516025
21415
507.2

Excessive output truncated after 524301 bytes.

507.5831657889654
21845
507.58392116225116
21846
507.584676582496
21847
507.58543204970204
21848
507.5861875638713
21849
507.58694312500586
21850
507.5876987331078
21851
507.5884543881792
21852
507.5892100902221
21853
507.5899658392386
21854
507.5907216352307
21855
507.59147747820055
21856
507.5922333681502
21857
507.59298930508163
21858
507.593745288997
21859
507.59450131989837
21860
507.5952573977878
21861
507.59601352266736
21862
507.5967696945391
21863
507.5975259134051
21864
507.5982821792674
21865
507.59903849212816
21866
507.5997948519894
21867
507.6005512588531
21868
507.60130771272145
21869
507.60206421359646
21870
507.60282076148025
21871
507.60357735637484
21872
507.60433399828236
21873
507.60509068720484
21874
507.6058474231443
21875
507.60660420610293
21876
507.6073610360827
21877
507.6081179130857
21878
507.60887483711406
21879
507.6096318081698
21880
507.610388826255
21881
507.61114589137173
21882
507.61190300352206
21883
507.6126601627081
21884
507.6134173689319
21885
5

In [113]:
X = [i * 4e-9 for i = 1:2000]
Yt_cant = py"np.load"("data/mass_fraction.npy")
plot(timestep, Yt')
#plot!(X,  Yt_cant')
xlabel!("Time")
ylabel!("Mass Fractions")
savefig("Mass_fraction.pdf")

In [183]:
plot(timestep, Yt')
xlabel!("Time")
ylabel!("Mass Fractions")
savefig("Mass_fraction.pdf")

In [184]:
X = [i * 4e-9 for i = 1:2000]
Tt_cant = py"np.load"("data/temperature.npy")
plot(timestep, Tt)
plot!(timestep[2:end], Tt_cant)
xlabel!("Time")
ylabel!("Temperature")
savefig("Temperature.pdf")

In [ ]:
X = [i * 4e-9 for i = 1:2000]
Pt_cant = py"np.load"("data/pressure.npy")
plot(timestep, Pt)
plot!(X, Pt_cant)
xlabel!("Time")
ylabel!("Pressure")

In [ ]:
plot(X, Tt)
Tt_cant = py"np.load"("data/temperature.npy")
plot(X, [Tt Tt_cant], label = ["Julia" "Cantera"])
xlabel!("Time")
ylabel!("Temperature")
#savefig("Temperautre.pdf")

In [29]:
Tt

2001-element Array{Float64,1}:
  500.0
  500.00029251171594
  500.00058502535046
  500.0008775409036
  500.0011700583754
  500.00146257776584
  500.00175509907496
  500.00204762230277
  500.00234014744933
  500.00263267451464
  500.00292520349876
  500.00321773440163
  500.00351026722336
    ⋮
 3051.9681894646483
 3051.930540591628
 3051.8929527967844
 3051.855425937356
 3051.8179598710167
 3051.780554455875
 3051.7432095504723
 3051.70592501378
 3051.6687007052
 3051.6315364845614
 3051.594432212119
 3051.557387748553

In [ ]:
dt = 1e-12
for i = 1:n_t
    k1 = f(TY)
    k2 = f(TY + 0.5 * dt * k1)
    k3 = f(TY + 0.5 * dt * k2)
    k4 = f(TY + dt * k3)
    TY += 1/6 * dt * (k1 + 2 * k2 + 2 * k3 + k4)
    #TY += dt * f(TY)
    Tt[i] = TY[1]
    print(TY[1])
    println(i)
    Y = TY[2:end]
    Yt[:,i] = Y
    X = ρ * Y ./ W
    p = sum(X) * R * TY[1]
    TYt[:,i] = TY
    Pt[i] = p
end

In [ ]:
function forward!(dty, ty, p, t)
    T = ty[1]
    Y = ty[2:end]
    ρ = m / V # density
    X = ρ * Y ./ W # Concentration
    Q = ones(M) # Individual progress rates
    # cpHS = compute_cpHS(NASA_coeffs, R, T)
    # cp = cpHS[:,1]
    # h = cpHS[:,2]
    # s = cpHS[:,3]
    #islarge = NASA_coeffs[:,1] .< T * ones(N)
    fhi = f_hi(NASA_coeffs, T, R)
    #flo = f_lo(NASA_coeffs, T, R)
    
#     cp = islarge .* fhi[:,1] + (1 .- islarge) .* flo[:,1]
#     h = islarge .* fhi[:,2] + (1 .- islarge) .* flo[:,2]
#     s = islarge .* fhi[:,3] + (1 .- islarge) .* flo[:,3]
    cp = fhi[:,1] 
    h = fhi[:,2] 
    s = fhi[:,3] 
    cvk = cp .- R
    ΔS = ν' * s  # Entropy change for reaction j
    ΔH = ν' * h # Entahlpy change for reaction j
    ####
    M_t = efficiency_t' * X
    Kf_t = Af_t .* (T .^ β_t) .* exp.(-E_t ./ (R * T)) .* M_t
    Kr_t = Kf_t ./ (((pa/(R * T)) .^ sum(ν[:,tbd], dims=1)' .* exp.(ΔS[tbd] ./ R - ΔH[tbd] ./ (R * T))))
    Qtbd = Kf_t .* prod(X .^ order_t, dims=1)' .- Kr_t .* prod(X .^ ν2[:,tbd], dims=1)' .* reversible[tbd]
    ####
    M_f = efficiency_f' * X
    Kf_lo = Af_lo .* (T .^ β_lo) .* exp.(-E_lo ./ (R * T)) .* M_f
    Kf_hi = Af_hi .* (T .^ β_hi) .* exp.(-E_hi ./ (R * T)) 
    Pr = Kf_lo ./ Kf_hi
    Fac = ones(size(falofr)[1])
    Fac[troefall] = [compute_falloff(T, Pr[s], troefall_coeff[:,i]) for (i,s) in enumerate(troefall)]
    Kf_f = Kf_lo ./ (1 .+ (Kf_lo ./ Kf_hi)) .* Fac
    Kr_f = Kf_f ./ (((pa/(R * T)) .^ sum(ν[:,falofr], dims=1)' .* exp.(ΔS[falofr] ./ R - ΔH[falofr] ./ (R * T))))
    Qfalofr = Kf_f .* prod(X .^ order_f, dims=1)' .- Kr_f .* prod(X .^ ν2[:,falofr], dims=1)' .* reversible[falofr]
    ####
    Kf = Af .* (T .^ β) .* exp.(-E ./ (R * T))
    Kr = Kf ./ (((pa/(R * T)) .^ sum(ν[:,elmr], dims=1)' .* exp.(ΔS[elmr] ./ R - ΔH[elmr] ./ (R * T))))
    Qelmr = Kf .* prod(X .^ order, dims=1)' .- Kr .* prod(X .^ ν2[:,elmr], dims=1)' .* reversible[elmr]
    ##### Computing ω_dot 
    cv = sum(cvk ./ W .* Y) # Mass heat capacities
    u = h ./ W - R ./ W .* T   # Internal energy for species
    p = sum(X) * R * T # pressure
    Q = [Qtbd' Qfalofr' Qelmr']
    ν_new = [ν[:,tbd] ν[:,falofr] ν[:,elmr]]
    ω_dot = W .* sum(ν_new .* Q, dims=2)
    ###### Species Conservation
    mgen_dot = V .* ω_dot
    dty[2:end] = (1 / m) .* (min_dot .* (Yin .- Y) - mout_dot .* Y .+ mgen_dot) 
    ###### Energy Conservation
    dty[1] = 1 / (m * cv) * (-Qdot + min_dot * (hin - sum(u .* Yin)) - p * V / m * mout_dot - sum(mgen_dot .* u))
end

In [ ]:
TY = [T;Y]
u0 = TY
tspan = (0.0,0.001)
prob = ODEProblem(forward!,u0,tspan)
sol = DifferentialEquations.solve(prob)

In [ ]:
using ModelingToolkit
de = modelingtoolkitize(prob)

In [ ]:
function f_hi(a, T, R)
        v1 = (a[:,2] + a[:,3] * T + a[:,4] * T^2 + a[:,5] * T^3 + a[:,6] * T^4) * R
        v2 = (a[:,2] + a[:,3] * T / 2 + a[:,4] * T^2 /3 + a[:,5] * T^3 /4 + a[:,6] * T^4 /5 + a[:,7]/T) * R * T
        v3 = (a[:,2] * log(T) + a[:,3] * T + a[:,4] * T^2 /2 + a[:,5] * T^3 /3 + a[:,6] * T^4 / 4 + a[:,8]) * R
    return [v1 v2 v3]
end
 
function f_lo(a, T, R)
    v1 = (a[:,9] + a[:,10] * T + a[:,11] * T^2 + a[:,12] * T^3 + a[:,13] * T^4) * R
    v2 = (a[:,9] + a[:,10] * T / 2 + a[:,11] * T^2 /3 + a[:,12] * T^3 /4 + a[:,13] * T^4 /5 + a[:,14]/T) * R * T
    v3 = (a[:,9] * log(T) + a[:,10] * T + a[:,11] * T^2 /2 + a[:,12] * T^3 /3 + a[:,13] * T^4 / 4 + a[:,15]) * R
    return [v1 v2 v3]
end 

In [ ]:
islarge = NASA_coeffs[:,1] .< T * ones(N)
fhi = f_hi(NASA_coeffs, T, R)
flo = f_lo(NASA_coeffs, T, R)
cp = islarge .* fhi[:,1] + (1 .- islarge) .* flo[:,1]
h = islarge .* fhi[:,2] + (1 .- islarge) .* flo[:,2]
s = islarge .* fhi[:,3] + (1 .- islarge) .* flo[:,3]

In [ ]:
Pt

In [ ]:
tf.isgreat(T, NASA_coeffs[:,1])

In [ ]:
tbd

In [ ]:
using DifferentialEquations

In [ ]:
log.(Af)

In [ ]:
log.(E)

In [ ]:
exp(20)

In [ ]:
Af

In [ ]:
E

In [ ]:
exp(17)

In [ ]:
exp(17.9616)

In [50]:
argmax(Tt)

19987

In [52]:
timestep[argmax(Tt)] #500K

0.001997300909999999

In [55]:
timestep[argmax(Tt_cant)]  #500K

0.0019967013599999995

In [77]:
timestep[argmax(Tt)] #800K

0.000107142841

In [78]:
timestep[argmax(Tt_cant)] #800K

0.000107128203

In [103]:
timestep[argmax(Tt)] # 700K

0.000208825266

In [104]:
timestep[argmax(Tt_cant)] #700K

0.00020880482999999998

In [119]:
ignition_1000 = timestep[argmax(Tt)] #1000K

3.8224395999999995e-5

In [120]:
ignition_1000_cant = timestep[argmax(Tt_cant)] #1000K

3.821565299999999e-5

In [130]:
ignition_1200 = timestep[argmax(Tt)] #1000K

1.6014970000000003e-5

In [131]:
ignition_1200_cant = timestep[argmax(Tt_cant)] #1000K

1.6009311000000003e-5

In [139]:
ignition_1500 = timestep[argmax(Tt)] #1500K

5.328479999999997e-6

In [140]:
ignition_1500_cant = timestep[argmax(Tt_cant)] #1500K

5.3252159999999965e-6

In [149]:
ignition_1800 = timestep[argmax(Tt)] #1500K

2.2503810000000006e-6

In [150]:
ignition_1800_cant = timestep[argmax(Tt_cant)] #1500K

2.2482600000000007e-6

In [165]:
ignition_600 = timestep[argmax(Tt)] #1500K

0.0005138737109999997

In [166]:
ignition_600_cant = timestep[argmax(Tt_cant)] #1500K

0.0005138416529999998

In [185]:
ignition_500 = timestep[argmax(Tt)] #1500K

0.001996962002999999

In [186]:
ignition_500_cant = timestep[argmax(Tt_cant)] #1500K

0.001996898805999999

In [195]:
Initial_Temp = [500, 600, 700, 800, 1000, 1200, 1500, 1800]
Ignition_Delay = [ignition_500, 0.0005138737109999997, 0.000208825266, 0.000107142841, ignition_1000, ignition_1200, ignition_1500, ignition_1800]

8-element Array{Float64,1}:
 0.001996962002999999
 0.0005138737109999997
 0.000208825266
 0.000107142841
 3.8224395999999995e-5
 1.6014970000000003e-5
 5.328479999999997e-6
 2.2503810000000006e-6

In [196]:
Ignition_Delay_Cant = [ignition_500_cant, 0.000513841652999, 0.00020880482999999998, 0.000107128203, ignition_1000_cant, ignition_1200_cant, ignition_1500_cant, ignition_1800_cant]

8-element Array{Float64,1}:
 0.001996898805999999
 0.000513841652999
 0.00020880482999999998
 0.000107128203
 3.821565299999999e-5
 1.6009311000000003e-5
 5.3252159999999965e-6
 2.2482600000000007e-6

In [208]:
plot(1000 ./ Initial_Temp, Ignition_Delay, seriestype = :scatter, label = "Julia", yaxis=:log)
plot!(1000 ./ Initial_Temp, Ignition_Delay, label = "Julia")
plot!(1000 ./ Initial_Temp, Ignition_Delay_Cant, label = "Cantera")
ylabel!("Ignition Delay Time")
xlabel!("1000 / Temperature")
savefig("Ignition_Delay.pdf")

In [ ]:
ignition_500